# Tutorial: Actor Critic Implementation

In [1]:
#Import required libraries

import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
#Set constants for training
seed = 543
log_interval = 10
gamma = 0.99

env = gym.make('CartPole-v1')
env.reset(seed=seed)
torch.manual_seed(seed)

SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
env = gym.make('CartPole-v1')
env.reset(seed=seed)
torch.manual_seed(seed)


SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])


class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)

        # actor's layer
        self.action_head = nn.Linear(128, 2)

        # critic's layer
        self.value_head = nn.Linear(128, 1)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.affine1(x))

        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)

        # critic: evaluates being in the state s_t
        state_values = self.value_head(x)

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t
        return action_prob, state_values


In [4]:
model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

In [5]:

def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)

    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))

    # the action to take (left or right)
    return action.item()


def finish_episode():
    """
    Training code. Calculates actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment

    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        # calculate actor (policy) loss
        policy_losses.append(-log_prob * advantage)

        # calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]


def train():
    running_reward = 10

    # run infinitely many episodes
    for i_episode in range(20000):

        # reset environment and episode reward
        state = env.reset()
        ep_reward = 0

        # for each episode, only run 9999 steps so that we don't
        # infinite loop while learning
        for t in range(1, 10000):

            # select action from policy
            action = select_action(state)

            # take the action
            state, reward, done, _ = env.step(action)

            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        # update cumulative reward
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

        # perform backprop
        finish_episode()

        # log results
        if i_episode % log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))

        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
train()

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 0	Last reward: 22.00	Average reward: 10.60
Episode 10	Last reward: 28.00	Average reward: 16.78
Episode 20	Last reward: 42.00	Average reward: 33.66
Episode 30	Last reward: 21.00	Average reward: 31.73
Episode 40	Last reward: 26.00	Average reward: 29.00
Episode 50	Last reward: 150.00	Average reward: 64.74
Episode 60	Last reward: 85.00	Average reward: 87.97
Episode 70	Last reward: 234.00	Average reward: 153.18
Episode 80	Last reward: 44.00	Average reward: 144.46
Episode 90	Last reward: 44.00	Average reward: 101.80
Episode 100	Last reward: 133.00	Average reward: 89.91
Episode 110	Last reward: 60.00	Average reward: 88.15
Episode 120	Last reward: 339.00	Average reward: 178.64
Episode 130	Last reward: 105.00	Average reward: 177.38
Episode 140	Last reward: 34.00	Average reward: 138.97
Episode 150	Last reward: 500.00	Average reward: 155.96
Episode 160	Last reward: 500.00	Average reward: 267.45
Episode 170	Last reward: 500.00	Average reward: 360.77
Episode 180	Last reward: 140.00	Average 

# TODO: Write a policy class similar to the above, without using shared features for the actor and critic and compare their performance.

In [7]:

#TODO: Write a policy class similar to the above, without using shared features for the actor and critic and compare their
# performance.
env = gym.make('CartPole-v1')
env.reset(seed=seed)
torch.manual_seed(seed)


SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])
class UnsharedPolicy(nn.Module):
  def __init__(self):
    super(UnsharedPolicy, self).__init__()

    #TODO: Fill in.
    self.actor_layer = nn.Linear(4, 128)
    self.critic_layer = nn.Linear(4, 128)
    self.action_head = nn.Linear(128, 2)
    self.value_head = nn.Linear(128, 1)
    self.saved_actions = []
    self.rewards = []

  def forward(self, x):
      # TODO: Fill in. For your networks, use the same hidden_size for the layers as the previous policy, that is 128.
      x_actor = F.relu(self.actor_layer(x))
      action_prob = F.softmax(self.action_head(x_actor), dim=-1)
      x_critic = F.relu(self.critic_layer(x))
      state_values = self.value_head(x_critic)
      return action_prob, state_values


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [8]:
model = UnsharedPolicy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()
train()

Episode 0	Last reward: 12.00	Average reward: 10.10
Episode 10	Last reward: 15.00	Average reward: 11.25
Episode 20	Last reward: 27.00	Average reward: 23.25
Episode 30	Last reward: 65.00	Average reward: 38.56
Episode 40	Last reward: 58.00	Average reward: 52.85
Episode 50	Last reward: 69.00	Average reward: 63.75
Episode 60	Last reward: 70.00	Average reward: 74.61
Episode 70	Last reward: 90.00	Average reward: 96.04
Episode 80	Last reward: 91.00	Average reward: 94.37
Episode 90	Last reward: 45.00	Average reward: 104.10
Episode 100	Last reward: 92.00	Average reward: 102.63
Episode 110	Last reward: 33.00	Average reward: 76.98
Episode 120	Last reward: 48.00	Average reward: 62.95
Episode 130	Last reward: 38.00	Average reward: 58.60
Episode 140	Last reward: 39.00	Average reward: 52.37
Episode 150	Last reward: 97.00	Average reward: 58.02
Episode 160	Last reward: 92.00	Average reward: 61.71
Episode 170	Last reward: 90.00	Average reward: 78.17
Episode 180	Last reward: 236.00	Average reward: 146.49


###Inference

As we can see, for the given hyper parameters,The shared parameters policy converges in less than 300 episodes(290), whereas unshared parameters- takes over 1000(1140)episodes to converge. \\
This is because in shared parameters, since both actor and critic have the same set of parameters shared, it cuts down opimising the parameters by twice as compared to unshared case, where the parameters are different and so they are optimised separately. Shared parameters also prevents optimising over parameters of actor and critic, which are redundant,ie both parameters are meant to be similar, thus cutting down redundant steps.

Hyperparameters also play a major role, we must use a better set of parameters for unshared case, as setting the same alpha value (learning rate) for actor and critic is not ideal.


###Experiment - decrase the learning rate and see




In [10]:
env = gym.make('CartPole-v1')
env.reset(seed=seed)
torch.manual_seed(seed)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


As we can see from the below, decreasing the learning rate helping in giving a smoother increase or convergence to optimal, as shown
But the number of episodes taken for the same is increased

In [12]:
model = UnsharedPolicy()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
eps = np.finfo(np.float32).eps.item()
train()

Episode 0	Last reward: 18.00	Average reward: 10.40
Episode 10	Last reward: 17.00	Average reward: 13.21
Episode 20	Last reward: 22.00	Average reward: 17.19
Episode 30	Last reward: 21.00	Average reward: 19.63
Episode 40	Last reward: 37.00	Average reward: 22.52
Episode 50	Last reward: 38.00	Average reward: 20.98
Episode 60	Last reward: 19.00	Average reward: 19.82
Episode 70	Last reward: 21.00	Average reward: 20.49
Episode 80	Last reward: 20.00	Average reward: 19.87
Episode 90	Last reward: 14.00	Average reward: 18.95
Episode 100	Last reward: 23.00	Average reward: 21.67
Episode 110	Last reward: 17.00	Average reward: 21.14
Episode 120	Last reward: 16.00	Average reward: 21.44
Episode 130	Last reward: 32.00	Average reward: 24.70
Episode 140	Last reward: 17.00	Average reward: 23.74
Episode 150	Last reward: 13.00	Average reward: 21.97
Episode 160	Last reward: 67.00	Average reward: 23.59
Episode 170	Last reward: 15.00	Average reward: 22.16
Episode 180	Last reward: 39.00	Average reward: 24.41
Epis